In [ ]:
!pip install bs4
!pip install lxml
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import time
!pip install requests beautifulsoup4

In [ ]:
all_data = pd.DataFrame()
for i in range(1, 251):
    page_id = i

    request = f'https://rest-app.net/api/ads?login=loskutova.stas@gmail.com&token=b51a9b7845b268b1f03567c3925775b8&page={page_id}&category_id=9'

    page = requests.get(request, headers ={'accept': 'application/json'})

    res = page.json()
    data = pd.DataFrame(res['data'])
    all_data = pd.concat([all_data, data], ignore_index=True)

    time.sleep(5)

all_data

,Id,url,avito_id,title,price,time,operator,phone,name,region,...,marka,model,engine_vol,transmission,year,body,coords,images_big,postfix,other
0,3102524705,hidden_in_demo,hidden_in_demo,"Mercedes-Benz C-класс 1.6 AT, 2014, 144 000 км",11999994,2025-03-01 20:50:12,,XXX,Елена,Свердловская область,...,Mercedes-Benz,C-класс,1.6,AT,2014,144 000 км,"{'lat': 56.797060492853, 'lng': 60.627241799724}",,Частное лицо,[]
1,3102524736,hidden_in_demo,hidden_in_demo,"JAECOO J7 1.6 AMT, 2024",14039400,2025-03-04 20:32:11,,XXX,JAECOO Мейли Авто Калининград,Калининградская область,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'lat': 54.706448, 'lng': 20.596178}",,Автодилер,[]
2,3102524789,hidden_in_demo,hidden_in_demo,"Ford Mondeo 2.3 AT, 2012, 158 000 км",4062000,2025-03-01 20:50:53,,XXX,ТОРГМАШ,Москва,...,Ford,Mondeo,2.3,AT,2012,158 000 км,"{'lat': 55.900441, 'lng': 37.632089}",,Автодилер,[]
3,3102524797,hidden_in_demo,hidden_in_demo,"Mercedes-Benz M-класс 3.5 AT, 2012, 280 000 км",1590000,2025-03-02 20:48:59,,XXX,Пользователь,Ульяновская область,...,Mercedes-Benz,M-класс,3.5,AT,2012,280 000 км,"{'lat': 54.35712814, 'lng': 48.37349701}",,Частное лицо,[]
4,3102524813,hidden_in_demo,hidden_in_demo,"Ford Mondeo 1.6 MT, 2011, 177 000 км",4800000,2025-03-01 20:48:59,,XXX,Роман,Ярославская область,...,Ford,,,,,,"{'lat': 56.73919296, 'lng': 38.85438156}",,Частное лицо,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5289,3102531551,hidden_in_demo,hidden_in_demo,"Mitsubishi Lancer 1.6 MT, 2005, 290 000 км",2970000,2025-03-03 21:30:11,,XXX,Пользователь,Тверская область,...,Mitsubishi,,,,,,"{'lat': 56.26301575, 'lng': 34.33441925}",,Частное лицо,[]
5290,3102531554,hidden_in_demo,hidden_in_demo,"ВАЗ (LADA) Granta 1.6 MT, 2024, 7 км",4316000,2025-03-01 21:30:09,,XXX,АЛЕКС-АВТО,Башкортостан,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'lat': 54.783243, 'lng': 56.073029}",,Автодилер,[]
5291,3102531856,hidden_in_demo,hidden_in_demo,"BMW X5 3.0 AT, 2023, 32 000 км",20000000,2025-03-02 21:27:09,,XXX,Пользователь,Челябинская область,...,BMW,X5,3.0,AT,2023,32 000 км,"{'lat': 55.170659583769, 'lng': 61.355114362729}",,Частное лицо,[]
5292,3102531857,hidden_in_demo,hidden_in_demo,"Renault Megane 1.6 MT, 2012, 127 910 км",4725000,2025-03-03 21:27:07,,XXX,"Группа компаний ""Интерпартнер""",Удмуртия,...,Renault,,,,,,"{'lat': 56.841798, 'lng': 53.122701}",,Автодилер,[]


In [ ]:
import re
# Функция для извлечения информации о машине
def extract_car_info(title):
    marka = None
    model = None
    engine_vol = None
    transmission = None
    year = None
    body = None

    parts = title.split(',')

    marka_model = parts[0].strip().split()
    if marka_model:
        marka = marka_model[0]   # Первое слово - марка
        model = " ".join(marka_model[1:])  # Остальные слова - модель

    year_match = re.search(r'(\d{4})', title)
    if year_match:
        year = year_match.group(1)

    engine_transmission_match = re.search(r'(\d+(\.\d+)?)(\s+(AT|MT))?', title)
    if engine_transmission_match:
        engine_vol = engine_transmission_match.group(1)
        if engine_transmission_match.group(4):
            transmission = engine_transmission_match.group(4)

    if len(parts) > 1:
        body = parts[1].strip()

    return {
        'marka': marka,
        'model': model,
        'engine_vol': engine_vol,
        'transmission': transmission,
        'year': year,
        'body': body
    }

extracted_data = pd.DataFrame()

if 'title' in all_data.columns:
    extracted_data = all_data['title'].apply(extract_car_info).apply(pd.Series)

final_data = pd.concat([all_data, extracted_data], axis=1)


print(final_data)

              Id             url        avito_id  \
0     3102524705  hidden_in_demo  hidden_in_demo   
1     3102524736  hidden_in_demo  hidden_in_demo   
2     3102524789  hidden_in_demo  hidden_in_demo   
3     3102524797  hidden_in_demo  hidden_in_demo   
4     3102524813  hidden_in_demo  hidden_in_demo   
...          ...             ...             ...   
5289  3102531551  hidden_in_demo  hidden_in_demo   
5290  3102531554  hidden_in_demo  hidden_in_demo   
5291  3102531856  hidden_in_demo  hidden_in_demo   
5292  3102531857  hidden_in_demo  hidden_in_demo   
5293  3102532396  hidden_in_demo  hidden_in_demo   

                                               title     price  \
0     Mercedes-Benz C-класс 1.6 AT, 2014, 144 000 км  11999994   
1                            JAECOO J7 1.6 AMT, 2024  14039400   
2               Ford Mondeo 2.3 AT, 2012, 158 000 км   4062000   
3     Mercedes-Benz M-класс 3.5 AT, 2012, 280 000 км   1590000   
4               Ford Mondeo 1.6 MT, 2011, 177

In [ ]:
columns_to_drop = ['phone', 'images_big', 'engine_vol', 'url', 'avito_id', 'body', 'coords', 'images' ]
final_data = final_data.drop(columns=columns_to_drop)

# Проверяем результат
print(final_data)

              Id                                           title     price  \
0     3102524705  Mercedes-Benz C-класс 1.6 AT, 2014, 144 000 км  11999994   
1     3102524736                         JAECOO J7 1.6 AMT, 2024  14039400   
2     3102524789            Ford Mondeo 2.3 AT, 2012, 158 000 км   4062000   
3     3102524797  Mercedes-Benz M-класс 3.5 AT, 2012, 280 000 км   1590000   
4     3102524813            Ford Mondeo 1.6 MT, 2011, 177 000 км   4800000   
...          ...                                             ...       ...   
5289  3102531551      Mitsubishi Lancer 1.6 MT, 2005, 290 000 км   2970000   
5290  3102531554            ВАЗ (LADA) Granta 1.6 MT, 2024, 7 км   4316000   
5291  3102531856                  BMW X5 3.0 AT, 2023, 32 000 км  20000000   
5292  3102531857         Renault Megane 1.6 MT, 2012, 127 910 км   4725000   
5293  3102532396                 Mazda 6 2.0 AT, 2022, 79 000 км   7000000   

                     time operator                            n

Продавцы


In [ ]:
unique_postfix = final_data['postfix'].unique()

num_unique_postfix = len(unique_postfix)

print(f'Количество уникальных postfix: {num_unique_postfix}')
print('Уникальные postfix:')
print(unique_postfix)

postfix_counts = final_data['postfix'].value_counts().reset_index()
postfix_counts.columns = ['postfix', 'количество']
print(postfix_counts)

Количество уникальных postfix: 3
Уникальные postfix:
['Частное лицо' 'Автодилер' 'Частное Лицо']
        postfix  количество
0  Частное лицо        3500
1     Автодилер        1503
2  Частное Лицо         291


In [ ]:
final_data['postfix'] = final_data['postfix'].replace('Частное Лицо', 'Частное лицо')

postfix_counts = final_data['postfix'].value_counts().reset_index()
postfix_counts.columns = ['postfix', 'количество']

print(postfix_counts)

        postfix  количество
0  Частное лицо        3791
1     Автодилер        1503


In [ ]:
import plotly.express as px

fig = px.pie(postfix_counts,
             values='количество',
             names='postfix',
             title='Распределение продавцов')

fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')

fig.show()


Вытаскиваем марки



In [ ]:
unique_brands = all_data['marka'].unique()

num_unique_brands = len(unique_brands)

print(f'Количество уникальных марок: {num_unique_brands}')
print('Уникальные марки:')
print(unique_brands)

Количество уникальных марок: 21
Уникальные марки:
['Mercedes-Benz' nan 'Ford' 'ГАЗ' 'Opel' 'Nissan' 'Kia' 'Chrysler'
 'Renault' 'Volkswagen' 'Skoda' 'Peugeot' 'Toyota' 'Hawtai' 'Honda'
 'Chevrolet' 'Mitsubishi' 'Chery' 'Audi' 'BMW' 'Mazda']


In [ ]:
brand_counts = all_data['marka'].value_counts().reset_index()
brand_counts.columns = ['marka', 'count']

In [ ]:
fig = px.bar(brand_counts, x='marka', y='count',
             title='Количество машин каждой марки',
             labels={'marka': 'Марка', 'count': 'Количество машин'},
             color='count',
             color_continuous_scale=px.colors.sequential.Viridis)

fig.update_xaxes(title_text='Марка', tickangle=45)
fig.update_yaxes(title_text='Количество машин')

fig.show()

In [ ]:
count_by_brand = all_data['marka'].value_counts().reset_index()
count_by_brand.columns = ['marka', 'count_m']

average_price_by_brand = all_data.groupby('marka')['price'].mean().reset_index()
average_price_by_brand.columns = ['marka', 'средняя_стоимость']

combined_data = pd.merge(count_by_brand, average_price_by_brand, on='marka')

fig = px.bar(combined_data, x='marka', y='средняя_стоимость',
             title='Средняя стоимость машин каждой марки (цвет по количеству машин)',
             labels={'marka': 'Марка', 'средняя_стоимость': 'Средняя стоимость'},
             color='count_m',
             color_continuous_scale=px.colors.sequential.Viridis)

fig.update_xaxes(title_text='Марка', tickangle=45)
fig.update_yaxes(title_text='Средняя стоимость')

fig.show()

In [ ]:
filtered_data = all_data[all_data['price'] <= 22000000]

count_by_brand = filtered_data['marka'].value_counts().reset_index()
count_by_brand.columns = ['marka', 'count_m']

average_price_by_brand = filtered_data.groupby('marka')['price'].mean().reset_index()
average_price_by_brand.columns = ['marka', 'средняя_стоимость']

combined_data = pd.merge(count_by_brand, average_price_by_brand, on='marka')

fig = px.bar(combined_data, x='marka', y='средняя_стоимость',
             title='Средняя стоимость машин каждой марки (цвет по количеству машин)',
             labels={'marka': 'Марка', 'средняя_стоимость': 'Средняя стоимость'},
             color='count_m',
             color_continuous_scale=px.colors.sequential.Viridis)

fig.update_xaxes(title_text='Марка', tickangle=45)
fig.update_yaxes(title_text='Средняя стоимость')

fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')


fig.show()


In [ ]:
import plotly.express as px
import plotly.graph_objects as go


fig = px.box(all_data,
             x='price',
             title='Box Plot price',
             labels={'price': 'Цена'},
             template='plotly_white')




fig.show()

Регионы



In [ ]:
unique_region = final_data['region'].unique()

num_unique_region = len(unique_region)

print(f'Количество уникальных region: {num_unique_region}')
print('Уникальные region:')
print(unique_region)

Количество уникальных region: 26
Уникальные region:
['Свердловская область' 'Калининградская область' 'Москва'
 'Ульяновская область' 'Ярославская область' 'Ставропольский край'
 'Белгородская область' 'Брянская область' 'Орловская область'
 'Воронежская область' 'Удмуртия' 'Татарстан' 'Калужская область'
 'Красноярский край' 'Курская область' 'Челябинская область'
 'Самарская область' 'Московская область' 'Ивановская область'
 'Краснодарский край' 'Нижегородская область' 'Новгородская область'
 'Адыгея' 'Тверская область' 'Башкортостан' 'Ханты-Мансийский АО']


In [ ]:
region_counts = all_data['region'].value_counts().reset_index()
region_counts.columns = ['region', 'количество']

In [ ]:
fig = px.bar(region_counts, x='region', y='количество',
             title='Количество машин region',
             labels={'region': 'region', 'количество': 'Количество машин'},
             color='количество',
             color_continuous_scale=px.colors.sequential.Viridis)

# Обновляем стиль графика
fig.update_xaxes(title_text='region', tickangle=45)
fig.update_yaxes(title_text='Количество машин')

fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')


# Показываем график
fig.show()

In [ ]:

fig = px.pie(region_counts,
             names='region',
             values='количество',
             title='Количество машин по регионам',
             labels={'region': 'Регион', 'количество': 'Количество машин'},
             color='количество')

# Обновляем стиль графика
fig.update_traces(textinfo='percent+label')  # Показываем процент и название на сегментах
fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')

# Показываем график
fig.show()


In [ ]:
count_by_region = all_data['region'].value_counts().reset_index()
count_by_region.columns = ['region', 'количество']

average_price_by_region = all_data.groupby('region')['price'].mean().reset_index()
average_price_by_region.columns = ['region', 'средняя_стоимость']

combined_data = pd.merge(count_by_region, average_price_by_region, on='region')

fig = px.bar(combined_data, x='region', y='средняя_стоимость',
             title='Средняя стоимость машины по региону (цвет по количеству машин)',
             labels={'region': 'region', 'средняя_стоимость': 'Средняя стоимость'},
             color='количество',
             color_continuous_scale=px.colors.sequential.Viridis)

fig.update_xaxes(title_text='region', tickangle=45)
fig.update_yaxes(title_text='Средняя стоимость')

fig.show()

In [ ]:

large_cities = ['Москва', 'Московская область', 'Челябинская область', 'Свердловская область', 'Татарстан', 'Краснодарский край']

region_counts['city_type'] = region_counts['region'].apply(lambda x: 'Крупные города' if x in large_cities else 'Другие города')

city_counts = region_counts.groupby('city_type')['количество'].sum().reset_index()

fig = px.pie(city_counts,
             names='city_type',
             values='количество',
             title='Количество машин в крупных городах и остальных',
             labels={'city_type': 'Тип города', 'количество': 'Количество машин'},
             color='количество')

fig.update_traces(textinfo='percent+label')  # Показываем процент и название на сегментах
fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')

# Показываем график
fig.show()


In [ ]:
fig = px.bar(region_seller_counts,
             x='region',
             y='количество',
             color='postfix',  # Цвет по типу продавца
             title='Количество машин в регионах, продаваемых частными лицами и автодилерами',
             labels={'region': 'Регион', 'количество': 'Количество машин'},
             barmode='group')  # Группировка по продавцу

# Обновляем стиль графика
fig.update_xaxes(title_text='Регион', tickangle=45)
fig.update_yaxes(title_text='Количество машин')

# Показываем график
fig.show()

In [ ]:
mean_score = all_data['price'].mean()
std_dev_score = all_data['price'].std()
min_score = all_data['price'].min()
max_score = all_data['price'].max()
median_score = all_data['price'].median()

print(f"Среднее значение рейтинга: {mean_score}")
print(f"Стандартное отклонение рейтинга: {std_dev_score}")
print(f"Минимальное значение рейтинга: {min_score}")
print(f"Максимальное значение рейтинга: {max_score}")
print(f"Медиана рейтинга: {median_score}")

Среднее значение рейтинга: 7169560.534945221
Стандартное отклонение рейтинга: 7278553.414054636
Минимальное значение рейтинга: 40000
Максимальное значение рейтинга: 55458000
Медиана рейтинга: 5250000.0


Примерные диапазоны цен на 2025 год

Экономичный сегмент Этот сегмент включает в себя бюджетные автомобили, такие как Lada Granta, Kia Rio, Hyundai Solaris и другие модели отечественного производства и недорогие иномарки. В 2025 году цены на подержанные автомобили экономичного сегмента могут варьироваться до 1 миллиона рублей.

Средний низкий сегмент К этому сегменту относятся автомобили среднего класса, такие как Skoda Octavia, Toyota Corolla, Ford Focus и аналогичные модели. Цены на подержанные автомобили этого сегмента могут составлять от 1 миллиона до 2 миллионов рублей.

Средний высокий сегмент Здесь находятся более дорогие модели среднего класса, включая BMW 3 серии, Audi A4, Mercedes-Benz C-класса и другие. Диапазон цен на подержанные автомобили среднего высокого сегмента может колебаться от 2 до 3 миллионов рублей.

Премиальный сегмент Этот сегмент представлен автомобилями премиум-класса, такими как BMW 5 серии, Audi A6, Mercedes-Benz E-класса и подобные модели. Цена на подержанные автомобили премиального сегмента может находиться в диапазоне от 3 до 5 миллионов рублей.

Люкс сегмент Автомобили люкс-сегмента включают в себя модели высшего класса, такие как Bentley, Rolls-Royce, Maybach и другие. Подержанные автомобили этого сегмента могут стоить от 5 миллионов рублей и выше, вплоть до десятков миллионов рублей в зависимости от марки, модели и состояния автомобиля.

In [ ]:
def classify_car(price):
    if price < 1000000:
        return 'Экономичный сегмент'
    elif 1000000 <= price < 2000000:
        return 'Средний низкий сегмент'
    elif 2000000 <= price < 3000000:
        return 'Средний высокий сегмент'
    elif 3000000 <= price < 5000000:
        return 'Премиальный сегмент'
    elif  22000000 > price >= 5000000:
        return 'Люкс сегмент'

# Применяем классификацию
all_data['class'] = all_data['price'].apply(classify_car)

class_counts = all_data['class'].value_counts().reset_index()
class_counts.columns = ['class', 'количество']

fig = px.bar(class_counts,
             x='class',
             y='количество',
             title='Количество автомобилей по классам',
             labels={'class': 'Класс автомобиля', 'количество': 'Количество'},
             color='количество',
             color_continuous_scale=px.colors.sequential.Viridis)

fig.update_xaxes(title_text='Класс автомобиля')
fig.update_yaxes(title_text='Количество автомобилей')

fig.show()


In [ ]:
all_data['class'] = all_data['price'].apply(classify_car)

# Подсчитываем количество автомобилей в каждом классе
class_counts = all_data['class'].value_counts().reset_index()
class_counts.columns = ['class', 'количество']

# Строим круговую диаграмму с помощью Plotly
fig = px.pie(class_counts,
             values='количество',
             names='class',
             title='Распределение автомобилей по классам',
            )

# Показываем график
fig.show()



final_data['year'] = pd.to_numeric(final_data['year'], errors='coerce')

# Удаляем строки с NaN в 'year'
final_data = final_data.dropna(subset=['year'])

# Определяем новые и б/у автомобили
final_data['car_condition'] = final_data['year'].apply(lambda x: 'Новые' if x >= 2022 else 'Б/У')

# Подсчитываем количество новых и б/у автомобилей
condition_counts = final_data['car_condition'].value_counts().reset_index()
condition_counts.columns = ['condition', 'количество']

# Убедимся, что есть данные для построения диаграммы
if not condition_counts.empty:
    # Создаем круговую диаграмму
    fig = px.pie(condition_counts,
                 names='condition',
                 values='количество',
                 title='Количество новых и б/у автомобилей',
                 labels={'condition': 'Состояние автомобиля', 'количество': 'Количество'})

    # Обновляем стиль графика
    fig.update_traces(textinfo='percent+label')  # Показываем процент и название на сегментах
    fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')

    # Показываем график
    fig.show()
else:
    print("Нет данных для построения диаграммы.")


In [ ]:

all_data['year'] = pd.to_numeric(all_data['year'], errors='coerce')


all_data = all_data.dropna(subset=['year'])

all_data['car_condition'] = all_data['year'].apply(lambda x: 'Новые' if x >= 2022 else 'Б/У')


condition_counts = all_data['car_condition'].value_counts().reset_index()
condition_counts.columns = ['condition', 'количество']


if not condition_counts.empty:
    # Создаем круговую диаграмму
    fig = px.pie(condition_counts,
                 names='condition',
                 values='количество',
                 title='Количество новых и б/у автомобилей',
                 labels={'condition': 'Состояние автомобиля', 'количество': 'Количество'})

    # Обновляем стиль графика
    fig.update_traces(textinfo='percent+label')  # Показываем процент и название на сегментах
    fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')

    # Показываем график
    fig.show()
else:
    print("Нет данных для построения диаграммы.")


<ipython-input-49-c3b265ed7593>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

